In [30]:
import pandas as pd
import datetime
import os
import re

In [31]:
# Справочник надо предварительно заполнить чтобы подтянуть нужные отделения, врачи и топ - нетоп
# Справочник подструктура нужна только для хирургии, травматологии, УЗФД, не делим по ТО. Во всех остальных структурах надо тоже заполнить одной строкой лучше тем же названием
# Обязательно надо проверить все ли UF_USER из расписания в справочнике и врачей у которых в UF_USER прочерк (-)
# загрузили 4 файла из АУ_ВУ (брал по пол года) + расписание
# объединил все в один DataFrame, сделал merge справочника
# отдельно загрузил расписание


# определяем год
CURRENT_YEAR = 2020
LAST_YEAR = CURRENT_YEAR - 1

pattern = r'^ч\d{1}\.xlsx$' # паттерн для частей общего датафрейма без расписания

data = pd.DataFrame()

for file in os.listdir('.'):
    if re.match(pattern, file):
        data = data.append(pd.read_excel(file))
        
handbook = pd.read_excel('Справочник_врачи_отделения_топ.xlsx')

data = data.merge(handbook[['UF_USER', 'Структура', 'Врачи - название', 'Топ/Не топ', 'Подструктура']], left_on='UF_USER', right_on='UF_USER', how='left')

data = data.rename(columns={'Sum([Стоимость полная])': 'Выработка', 'Номер карты': 'Количество клиентов', 'Ключ посещения': 'Количество посещений'})

timetable = pd.read_excel('Расписание.xlsx')

timetable = timetable.merge(handbook[['UF_USER', 'Структура', 'Врачи - название', 'Топ/Не топ', 'Подструктура']], left_on='UF_USER', right_on='UF_USER', how='left')

### Проверка есть ли в датах приема прочерки (в датах первого посещения, обычно, они есть)

In [32]:
print(data[data['Дата приема'] == '-'])


Empty DataFrame
Columns: [Структура 2 уровень, Структура 3 уровень, Структура 4 уровень, UF_USER, Исполнитель, Филиал, Год, Внутренняя категория, Количество клиентов, МДВ, Наименование услуги, Категория крупно, Количество посещений, Дата приема, Возрастная группа, Минимальная дата приема Пациента (без удаленных), Категория клиента ПЭО, Sum([Количество]), Выработка, АУ/ВУ, Структура 2 уровень.1, Структура 2 уровень.2, Структура, Врачи - название, Топ/Не топ, Подструктура]
Index: []

[0 rows x 26 columns]


### Создадим столбец с пометкой новый или старый исходя из даты приема, даты первого посещения и года

In [33]:
def is_new_pacient(row):
    if row['Минимальная дата приема Пациента (без удаленных)'] == '-':
        return 'Старый'
    elif row['Минимальная дата приема Пациента (без удаленных)'].date().year < row['Дата приема'].date().year:
        return 'Старый'
    else:
        return 'Новый'
    
data['Новый/старый'] = data.apply(is_new_pacient, axis=1)
    

In [34]:
# создам объект Excel_writer для записи туда сводных

writer = pd.ExcelWriter('Сводные.xlsx')

## АУ+ВУ

In [35]:
temp_df = data.pivot_table(index=['Структура'], columns=['АУ/ВУ', 'Год'], 
                      values=['Выработка', 'Количество клиентов', 'Количество посещений'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique', 
                               'Количество посещений': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 2)
temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[True, False, True])
temp_df.to_excel(writer, sheet_name='АУ+ВУ')

## Принадлежности пациентов АУ

In [36]:
temp_df = data[(data['АУ/ВУ'] == 'АУ') & (data['Категория клиента ПЭО'].isin(['АБ РЕН', 'МЦ АБ', 'НАЛ', 'СК факт', 'Соц Пакет']))].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Категория клиента ПЭО'], 
                      values=['Выработка', 'Количество клиентов', 'Количество посещений'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique', 
                               'Количество посещений': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, True, True])
temp_df.to_excel(writer, sheet_name='Принадлежности пациентов АУ')

## Принадлежности пациентов ВУ

In [37]:
temp_df = data[(data['АУ/ВУ'] == 'ВУ') & (data['Категория клиента ПЭО'].isin(['АБ РЕН', 'МЦ АБ', 'НАЛ', 'СК факт', 'Соц Пакет']))].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Категория клиента ПЭО'], 
                      values=['Выработка', 'Количество клиентов', 'Количество посещений'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique', 
                               'Количество посещений': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, True, True])
temp_df.to_excel(writer, sheet_name='Принадлежности пациентов ВУ')

## Диаграмма выработка

In [38]:
# 1я таблица врачи-название

temp_df = data[data['Год'] == CURRENT_YEAR].groupby(['Врачи - название']).agg({'Выработка': sum, 'Количество клиентов': 'nunique'})
temp_df['Средний чек'] = temp_df['Выработка'] / temp_df['Количество клиентов']
temp_df.drop('Количество клиентов', axis=1, inplace=True)

grow_base = data.pivot_table(index=['Врачи - название'], columns=['Год'], 
                      values=['Выработка'], 
                      aggfunc={'Выработка': sum})

grow_base['Прирост'] = (grow_base[('Выработка', CURRENT_YEAR)] / grow_base[('Выработка', LAST_YEAR)]) - 1

temp_df = temp_df.merge(grow_base[['Прирост']], how='left', left_index=True, right_index=True)

temp_df = temp_df.rename(columns={('Прирост', ''): 'Прирост'})

temp_df = temp_df[['Выработка','Прирост', 'Средний чек']]

temp_df.to_excel(writer, sheet_name='Диаграмма выработка 1')





C:\Users\AmanovRA\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [39]:
# 2я таблица подструктура

temp_df = data[data['Год'] == CURRENT_YEAR].groupby(['Подструктура']).agg({'Выработка': sum, 'Количество клиентов': 'nunique'})
temp_df['Средний чек'] = temp_df['Выработка'] / temp_df['Количество клиентов']
temp_df.drop('Количество клиентов', axis=1, inplace=True)

grow_base = data.pivot_table(index=['Подструктура'], columns=['Год'], 
                      values=['Выработка'], 
                      aggfunc={'Выработка': sum})

grow_base['Прирост'] = (grow_base[('Выработка', CURRENT_YEAR)] / grow_base[('Выработка', LAST_YEAR)]) - 1

temp_df = temp_df.merge(grow_base[['Прирост']], how='left', left_index=True, right_index=True)

temp_df = temp_df.rename(columns={('Прирост', ''): 'Прирост'})

temp_df = temp_df[['Выработка','Прирост', 'Средний чек']]

temp_df.to_excel(writer, sheet_name='Диаграмма выработка 2')

C:\Users\AmanovRA\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


## Категории возрастов АУ

In [40]:
# 1я таблица Количество пациентов

def sort_ages(x):
    pattern = r'[- ]'
    
    
    if all([isinstance(i, (int, float)) for i in x]):
        return [i for i in x]
    else:
        return [int(re.split(pattern, i)[0]) if i != 'oshibka' else 200 for i in x]
    

temp_df = data[data['АУ/ВУ'] == 'АУ'].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Возрастная группа'], 
                      values=['Количество клиентов'], 
                      aggfunc={'Количество клиентов': 'nunique'})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True], key = sort_ages)
temp_df.to_excel(writer, sheet_name='Категории возрастов АУ 1')

In [41]:
# 2я таблица Выработка

def sort_ages(x):
    pattern = r'[- ]'
    
    
    if all([isinstance(i, (int, float)) for i in x]):
        return [i for i in x]
    else:
        return [int(re.split(pattern, i)[0]) if i != 'oshibka' else 200 for i in x]
    

temp_df = data[data['АУ/ВУ'] == 'АУ'].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Возрастная группа'], 
                      values=['Выработка'], 
                      aggfunc={'Выработка': sum})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True], key = sort_ages)
temp_df.to_excel(writer, sheet_name='Категории возрастов АУ 2')

In [42]:
# 3я таблица Количество посещений

def sort_ages(x):
    pattern = r'[- ]'
    
    
    if all([isinstance(i, (int, float)) for i in x]):
        return [i for i in x]
    else:
        return [int(re.split(pattern, i)[0]) if i != 'oshibka' else 200 for i in x]
    

temp_df = data[data['АУ/ВУ'] == 'АУ'].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Возрастная группа'], 
                      values=['Количество посещений'], 
                      aggfunc={'Количество посещений': 'nunique'})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True], key = sort_ages)
temp_df.to_excel(writer, sheet_name='Категории возрастов АУ 3')

## Категории возрастов ВУ

In [43]:
# 1я таблица Количество пациентов

def sort_ages(x):
    pattern = r'[- ]'
    
    
    if all([isinstance(i, (int, float)) for i in x]):
        return [i for i in x]
    else:
        return [int(re.split(pattern, i)[0]) if i != 'oshibka' else 200 for i in x]
    

temp_df = data[data['АУ/ВУ'] == 'ВУ'].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Возрастная группа'], 
                      values=['Количество клиентов'], 
                      aggfunc={'Количество клиентов': 'nunique'})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True], key = sort_ages)
temp_df.to_excel(writer, sheet_name='Категории возрастов ВУ 1')

In [44]:
# 2я таблица Выработка

def sort_ages(x):
    pattern = r'[- ]'
    
    
    if all([isinstance(i, (int, float)) for i in x]):
        return [i for i in x]
    else:
        return [int(re.split(pattern, i)[0]) if i != 'oshibka' else 200 for i in x]
    

temp_df = data[data['АУ/ВУ'] == 'ВУ'].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Возрастная группа'], 
                      values=['Выработка'], 
                      aggfunc={'Выработка': sum})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True], key = sort_ages)
temp_df.to_excel(writer, sheet_name='Категории возрастов ВУ 2')

In [45]:
# 3я таблица Количество посещений

def sort_ages(x):
    pattern = r'[- ]'
    
    
    if all([isinstance(i, (int, float)) for i in x]):
        return [i for i in x]
    else:
        return [int(re.split(pattern, i)[0]) if i != 'oshibka' else 200 for i in x]
    

temp_df = data[data['АУ/ВУ'] == 'ВУ'].pivot_table(index=['Структура', 'Подструктура'], columns=['Год', 'Возрастная группа'], 
                      values=['Количество посещений'], 
                      aggfunc={'Количество посещений': 'nunique'})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True], key = sort_ages)
temp_df.to_excel(writer, sheet_name='Категории возрастов ВУ 3')

## Новые пациенты

In [46]:
# 1я таблица все АУ

temp_df = data[data['АУ/ВУ'] == 'АУ'].pivot_table(index=['Структура'], columns=['Год', 'Новый/старый'], 
                      values=['Выработка', 'Количество клиентов'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 2)
temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, True, True])
temp_df.to_excel(writer, sheet_name='Новые пациенты все АУ')

In [47]:
# 2я таблица НАЛ АУ

temp_df = data[(data['АУ/ВУ'] == 'АУ') & (data['Категория клиента ПЭО'] == 'НАЛ')].pivot_table(index=['Структура'], columns=['Год', 'Новый/старый'], 
                      values=['Выработка', 'Количество клиентов'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 2)
temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, True, True])
temp_df.to_excel(writer, sheet_name='Новые пациенты НАЛ АУ')

In [48]:
# 3я таблица СК АУ

temp_df = data[(data['АУ/ВУ'] == 'АУ') & (data['Категория клиента ПЭО'] == 'СК факт')].pivot_table(index=['Структура'], columns=['Год', 'Новый/старый'], 
                      values=['Выработка', 'Количество клиентов'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 2)
temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, True, True])
temp_df.to_excel(writer, sheet_name='Новые пациенты СК АУ')

## Категории исполнителей

In [49]:
temp_df = data.pivot_table(index=['Структура'], columns=['Год', 'Топ/Не топ'], 
                      values=['Выработка', 'Количество клиентов'], 
                      aggfunc={'Выработка': sum,
                                'Количество клиентов': 'nunique'})

temp_df.columns = temp_df.columns.swaplevel(0, 2)
temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, False, True])
temp_df.to_excel(writer, sheet_name='Категории исполнителей')

## Диаграмма расписание

In [50]:
# 1я таблица врачи-название

temp_df = timetable[timetable['Год'] == CURRENT_YEAR].pivot_table(index=['Врачи - название'], 
                      values=['Доступно для записи Пациентов', 'всего время пациентов', 'UF_USER'], 
                      aggfunc={'Доступно для записи Пациентов': sum,
                                'всего время пациентов': sum, 
                               'UF_USER': 'nunique'})

temp_df['Заполняемость'] = temp_df['всего время пациентов'] / temp_df['Доступно для записи Пациентов']
temp_df['План (тысяч часов)'] = (temp_df['Доступно для записи Пациентов'] * 24 / 1000).astype(int)
temp_df = temp_df.rename(columns = {'UF_USER': 'Исполнителей'})
temp_df = temp_df[['План (тысяч часов)', 'Заполняемость', 'Исполнителей']]
temp_df.to_excel(writer, sheet_name='Диаграмма расписание 1 врачи')

In [51]:
# 2я таблица подструктура

temp_df = timetable[timetable['Год'] == CURRENT_YEAR].pivot_table(index=['Подструктура'], 
                      values=['Доступно для записи Пациентов', 'всего время пациентов', 'UF_USER'], 
                      aggfunc={'Доступно для записи Пациентов': sum,
                                'всего время пациентов': sum, 
                               'UF_USER': 'nunique'})

temp_df['Заполняемость'] = temp_df['всего время пациентов'] / temp_df['Доступно для записи Пациентов']
temp_df['План (тысяч часов)'] = (temp_df['Доступно для записи Пациентов'] * 24 / 1000).astype(int)
temp_df = temp_df.rename(columns = {'UF_USER': 'Исполнителей'})
temp_df = temp_df[['План (тысяч часов)', 'Заполняемость', 'Исполнителей']]
temp_df.to_excel(writer, sheet_name='Диаграмма расписание 2 под-ура')

## Филиалы

In [52]:
# 1я таблица выработка по филиалам

temp_df = data[data['Филиал'] != 'БДВУ'].pivot_table(index=['Структура'], columns=['Год', 'Филиал'], 
                      values=['Выработка'], 
                      aggfunc={'Выработка': sum})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True])
temp_df.to_excel(writer, sheet_name='Филиалы 1 выработка')

In [53]:
# 2я таблица плановые часы по филиалам

temp_df = timetable[timetable['Филиал'] != 'ВУ'].pivot_table(index=['Структура'], columns=['Год', 'Филиал'], 
                      values=['Доступно для записи Пациентов'], 
                      aggfunc={'Доступно для записи Пациентов': sum})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True])
temp_df.to_excel(writer, sheet_name='Филиалы 2 плановые часы')

In [54]:
# 3я таблица фактические часы по филиалам

temp_df = timetable[timetable['Филиал'] != 'ВУ'].pivot_table(index=['Структура'], columns=['Год', 'Филиал'], 
                      values=['всего время пациентов'], 
                      aggfunc={'всего время пациентов': sum})

temp_df.sort_index(axis=1, inplace=True, level=[1, 2], ascending=[False, True])
temp_df.to_excel(writer, sheet_name='Филиалы 3 фактические часы')

## Диаграмма филиалы

In [55]:
temp_df = data[data['Год'] == CURRENT_YEAR].groupby('Филиал').agg({'Выработка': sum, 'Структура 4 уровень': 'nunique'}).merge(timetable.groupby('Филиал').agg({'Доступно для записи Пациентов': sum,
                                                                                                                       'всего время пациентов': sum}), left_index=True, right_index=True)

temp_df['Выработка млн. руб.'] = (temp_df['Выработка'] / 1000000).astype(int)
temp_df['Заполняемость'] = temp_df['всего время пациентов'] / temp_df['Доступно для записи Пациентов']
temp_df['Отделений'] = temp_df['Структура 4 уровень']

temp_df = temp_df[['Выработка млн. руб.', 'Заполняемость', 'Отделений']]

temp_df.to_excel(writer, sheet_name='Диаграмма филиалы')

## Филиалы топы

In [56]:
temp_df = timetable.pivot_table(index=['Филиал'], columns=['Год', 'Топ/Не топ'], 
                      values=['Доступно для записи Пациентов', 'всего время пациентов'], 
                      aggfunc={'Доступно для записи Пациентов': sum,
                                'всего время пациентов': sum})
def sort_timetable(x):
    
    if 'Доступно для записи Пациентов' in x:
        return [['Доступно для записи Пациентов', 'всего время пациентов', 'Заполняемость'].index(i) for i in x]
    else:
        return [i for i in x]


for column in temp_df.columns:
    if column[0] == 'Доступно для записи Пациентов':
        temp_df[('Заполняемость',) + column[1:]] = temp_df[('всего время пациентов',) + column[1:]] / temp_df[column]

temp_df.columns = temp_df.columns.swaplevel(0, 2)
temp_df.columns = temp_df.columns.swaplevel(0, 1)
temp_df.sort_index(axis=1, inplace=True, level=[0, 1, 2], ascending=[False, False, True], key=sort_timetable)
temp_df.to_excel(writer, sheet_name='Филиалы топы')



In [57]:
writer.save()